In [2]:
#Import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import io
import time
import datetime
import time
plt.style.use("ggplot")

In [3]:
data = pd.read_excel('card transactions.xlsx')
#Keep only type P transactions
data=data[data.Transtype=='P']

In [4]:
data.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

In [117]:
data.describe()

,Recnum,Cardnum,Merch zip,Amount,Fraud
count,96398.000,96398.000,92097.000,96398.000,96398.000
mean,48365.527,5142201666.932,44706.597,428.034,0.011
std,27944.862,55657.980,28369.538,10024.458,0.104
min,1.000,5142110002.000,1.000,0.010,0.000
25%,24154.250,5142152067.000,20855.000,33.460,0.000
50%,48365.500,5142195612.000,38118.000,137.980,0.000
75%,72577.750,5142245297.000,63103.000,428.250,0.000
max,96753.000,5142847398.000,99999.000,3102045.530,1.000


In [118]:
data.shape

(96398, 10)

In [5]:
#Filter out FEDEX
DFwithoutFEDEX = data[(data['Merch description'].str.lower().str.contains('fedex') == False)]
DFwithoutFEDEX.shape

(84623, 10)

In [6]:
#Get the first digit
DFwithoutFEDEX['First_Digit'] = DFwithoutFEDEX['Amount'].multiply(100).apply(lambda row: str(row)[0])
#Doublecheck
DFwithoutFEDEX['First_Digit'].value_counts()

C:\Users\vuduo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1    25697
2    15827
3    10298
4     7686
5     6749
6     5469
7     4699
8     4152
9     4046
Name: First_Digit, dtype: int64

In [7]:
#Define the function
def benfordstat(series, n_mid=15, c=3):
    count_low=sum(pd.to_numeric(series)<=2)
    count_all=len(series)
    r=count_low/count_all/0.477
    t=(count_all-n_mid)/c
    new_r=1+(r-1)/(1+np.exp(-t))
    stat=max(new_r, 1/new_r)
    return(stat)
    

In [8]:
#MERCHNUM
MN_stat = DFwithoutFEDEX.groupby(['Merchnum'])['First_Digit'].apply(benfordstat).reset_index()
MN_stat.columns=['Merchnum', 'BenFordStat']
MN_stat.head(10)

C:\Users\vuduo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


,Merchnum,BenFordStat
0,0,1.067996
1,003100006NOT6,1.009404
2,004740006ABC6,1.010214
3,005590006PNB6,1.009404
4,014430619 14,1.010214
5,014938913 51,1.009404
6,014948837 52,1.013124
7,014988879 18,1.009404
8,016560006NOT0,1.009404
9,017670006NOT0,1.001202


In [9]:
#CARDNUM
CN_stat = DFwithoutFEDEX.groupby(['Cardnum'])['First_Digit'].apply(benfordstat).reset_index()
CN_stat.columns=['Cardnum', 'BenFordStat']
CN_stat.head(10)

,Cardnum,BenFordStat
0,5142110002,1.010214
1,5142110081,1.025562
2,5142110313,1.007152
3,5142110402,1.098099
4,5142110434,1.010214
5,5142110651,1.018316
6,5142110691,1.149137
7,5142110749,1.013124
8,5142110909,1.549873
9,5142111097,1.059953


In [124]:
CN_stat.sort_values(by = 'BenFordStat', ascending = False).head(40).to_csv('Benford_Cardnum.csv')
MN_stat.sort_values(by = 'BenFordStat', ascending = False).head(40).to_csv('Benford_Merchnum.csv')

In [10]:
MN_stat.sort_values(by = 'BenFordStat', ascending = False).head(1)

,Merchnum,BenFordStat
12842,991808369338,inf


In [12]:
DFwithoutFEDEX[DFwithoutFEDEX['Merchnum']=='991808369338'][['Recnum','Cardnum', 'Date', 'Amount', 'First_Digit']]

,Recnum,Cardnum,Date,Amount,First_Digit
57,58,5142197563,2010-01-02,30.00,3
170,171,5142197563,2010-01-03,30.00,3
1129,1130,5142197563,2010-01-07,30.00,3
1734,1735,5142197563,2010-01-10,30.00,3
3706,3707,5142197563,2010-01-18,30.00,3
5266,5267,5142197563,2010-01-24,30.00,3
5949,5950,5142197563,2010-01-27,30.00,3
6403,6404,5142197563,2010-01-28,30.00,3
8258,8259,5142197563,2010-02-04,30.00,3
9478,9479,5142197563,2010-02-09,30.00,3
